![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.Pretrained_Clinical_Pipelines.ipynb)

# 11. Pretrained_Clinical_Pipelines

## Colab Setup

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.2.3
Spark NLP_JSL Version : 5.2.2


## Listing Models, Pipelines and Annotators

**You can print the list of clinical pretrained models/pipelines and annotators in Spark NLP with one-line code:**

In [ ]:
from sparknlp_jsl.pretrained import InternalResourceDownloader

# print PretrainedPipelines
InternalResourceDownloader.showPrivatePipelines(lang='en')

# print models
#InternalResourceDownloader.showPrivateModels(annotator="MedicalNerModel", lang='en')

# print annotators
# InternalResourceDownloader.showAvailableAnnotators()

+--------------------------------------------------------------+------+---------+
| Pipeline                                                     | lang | version |
+--------------------------------------------------------------+------+---------+
| clinical_analysis                                            |  en  | 2.4.0   |
| clinical_ner_assertion                                       |  en  | 2.4.0   |
| clinical_deidentification                                    |  en  | 2.4.0   |
| explain_clinical_doc_ade                                     |  en  | 2.7.3   |
| recognize_entities_posology                                  |  en  | 3.0.0   |
| explain_clinical_doc_carp                                    |  en  | 3.0.0   |
| explain_clinical_doc_ade                                     |  en  | 3.0.0   |
| explain_clinical_doc_era                                     |  en  | 3.0.0   |
| icd10cm_snomed_mapping                                       |  en  | 3.0.2   |
| snomed_icd10cm

## Pretrained Pipelines

In order to save you from creating a pipeline from scratch, Spark NLP also has a pre-trained pipelines that are already fitted using certain annotators and transformers according to various use cases.

**1.   explain_clinical_doc_granular** :

> A pipeline with `ner_jsl`, `assertion_jsl` and `re_test_result_date`. It will extract clinical and medication entities, assign assertion status and find relationships between clinical entities.

**2.   explain_clinical_doc_carp** :

> A pipeline with `ner_clinical`, `assertion_dl`, `re_clinical` and `ner_posology`. It will extract clinical and medication entities, assign assertion status and find relationships between clinical entities.

**3.   explain_clinical_doc_era** :

> A pipeline with `ner_clinical_events`, `assertion_dl` and `re_temporal_events_clinical`. It will extract clinical entities, assign assertion status and find temporal relationships between clinical entities.

**4.   explain_clinical_doc_ade** :

> A pipeline for `Adverse Drug Events (ADE)` with `ner_ade_biobert`, `assertiondl_biobert`, `classifierdl_ade_conversational_biobert` and `re_ade_biobert`. It will classify the document, extract `ADE` and `DRUG` entities, assign assertion status to `ADE` entities, and relate them with `DRUG` entities, then assign ADE status to a text (`True` means ADE, `False` means not related to ADE).

**letter codes in the naming conventions:**

> c : ner_clinical

> e : ner_clinical_events

> r : relation extraction

> p : ner_posology

> a : assertion

> ade : adverse drug events

**Relation Extraction types:**

`re_clinical` >> TrIP (improved), TrWP (worsened), TrCP (caused problem), TrAP (administered), TrNAP (avoided), TeRP (revealed problem), TeCP (investigate problem), PIP (problems related)

`re_temporal_events_clinical` >> `AFTER`, `BEFORE`, `OVERLAP`

**5. explain_clinical_doc_medication:**

> A pipeline for detecting posology entities with the `ner_posology_large` NER model, assigning their assertion status with `assertion_jsl` model, and extracting relations between posology-related terminology with `posology_re` relation extraction model.


**6. explain_clinical_doc_radiology**

> A pipeline for detecting radiology entities with the `ner_radiology` NER model, assigning their assertion status with `assertion_dl_radiology` model, and extracting relations between the diagnosis, test, and findings with `re_test_problem_finding` relation extraction model.

**7. Clinical Deidentification** :

>This pipeline can be used to deidentify PHI information from medical texts. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate `AGE`, `CONTACT`, `DATE`, `ID`, `LOCATION`, `NAME`, `PROFESSION`, `CITY`, `COUNTRY`, `DOCTOR`, `HOSPITAL`, `IDNUM`, `MEDICALRECORD`, `ORGANIZATION`, `PATIENT`, `PHONE`, `PROFESSION`, `STREET`, `USERNAME`, `ZIP`, `ACCOUNT`, `LICENSE`, `VIN`, `SSN`, `DLN`, `PLATE`, `IPADDR` entities.

**8.   explain_clinical_doc_generic** :

> A pipeline with `ner_clinical`, `assertion_dl` and `re_clinical`. It will extract clinical and medication entities, assign assertion status and find relationships between clinical entities.

- Clinical Entity Labels: `PROBLEM`, `TEST`, `TREATMENT`

- Assertion Status Labels: `Present`, `Absent`, `Possible`, `Planned`, `Past`, `Family`, `Hypotetical`, `SomeoneElse`

- Relation Extraction Labels: `TrAP`, `TeRP`, `TrIP`, `TrWP`, `TrCP`, `TrAP`, `TrNAP`, `TeCP`, `PIP`

**9.   explain_clinical_doc_oncology** :

> Pipelines includes Named-Entity Recognition, Assertion Status and Relation Extraction models to extract information from oncology texts.


**10.   explain_clinical_doc_vop** :

> Pipelines includes Named-Entity Recognition, Assertion Status, Relation Extraction and Entity Resolution models to extract information from clinical texts.

**11. NER Pipelines:**

> Pipelines for all the available pretrained NER models.

**12. BERT Based NER Pipelines**

> Pipelines for all the available Bert token classification models.

**13. ner_profiling_clinical and ner_profiling_biobert:**

> Pipelines for exploring all the available pretrained NER models at once.

**14. ner_model_finder**

> A pipeline trained with bert embeddings that can be used to find the most appropriate NER model given the entity name.

**15. Resolver Pipelines**

> Pipelines for converting clinical entities to their UMLS CUI codes and medication entities to their ADE, Action, Treatment, UMLS, RxNorm, ICD9, SNOMED and NDC codes.

**16. Oncology Pipelines**

> Pipelines includes Named-Entity Recognition, Assertion Status, Relation Extraction and Entity Resolution models to extract information from oncology texts.


**Also, you can find clinical CODE MAPPING pretrained pipelines in this notebook: [Healthcare Code Mapping Notebook](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/11.1.Healthcare_Code_Mapping.ipynb)**





## Task Based Pretrained Pipelines


|index|pipeline|index|pipeline|
|-----:|:-----|-----:|:-----|
| 1| [explain_clinical_doc_generic](https://nlp.johnsnowlabs.com/2024/01/17/explain_clinical_doc_generic_en.html)  | 2| [explain_clinical_doc_granular](https://nlp.johnsnowlabs.com/2024/01/26/explain_clinical_doc_granular_en.html)  |
|3| [clinical_deidentification](https://nlp.johnsnowlabs.com/2024/01/10/clinical_deidentification_en.html)  | 4| [explain_clinical_doc_oncology](https://nlp.johnsnowlabs.com/2024/01/18/explain_clinical_doc_oncology_en.html)  |
| 5| [explain_clinical_doc_radiology](https://nlp.johnsnowlabs.com/2024/01/18/explain_clinical_doc_radiology_en.html)  | 6| [explain_clinical_doc_vop](https://nlp.johnsnowlabs.com/2024/01/16/explain_clinical_doc_vop_en.html)
| 7| [explain_clinical_doc_carp](https://nlp.johnsnowlabs.com/2023/06/17/explain_clinical_doc_carp_en.html)  | 8| [explain_clinical_doc_era](https://nlp.johnsnowlabs.com/2023/06/17/explain_clinical_doc_era_en.html)  |
| 9| [explain_clinical_doc_ade](https://nlp.johnsnowlabs.com/2023/06/17/explain_clinical_doc_ade_en.html)| 10| [explain_clinical_doc_medication](https://nlp.johnsnowlabs.com/2023/06/17/explain_clinical_doc_medication_en.html)|




### explain_clinical_doc_granular

A pipeline with ner_jsl, assertion_jsl and re_test_result_date. It will extract clinical and medication entities, assign assertion status and find relationships between clinical entities.

- Clinical Entity Labels: `Admission_Discharge`, `Age`, `Alcohol`, `Allergen`, `BMI`, `Birth_Entity`, `Blood_Pressure`, `Cerebrovascular_Disease`, `Clinical_Dept`, `Communicable_Disease`, `Date`, `Death_Entity`, `Diabetes`, `Diet`, `Direction`, `Disease_Syndrome_Disorder`, `Dosage`, `Drug_BrandName`, `Drug_Ingredient`, `Duration`, `EKG_Findings`, `Employment`, `External_body_part_or_region`, `Family_History_Header`, `Fetus_NewBorn`, `Form`, `Frequency`, `Gender`, `HDL`, `Heart_Disease`, `Height`, `Hyperlipidemia`, `Hypertension`, `ImagingFindings`, `Imaging_Technique`, `Injury_or_Poisoning`, `Internal_organ_or_component`, `Kidney_Disease`, `LDL`, `Labour_Delivery`, `Medical_Device`, `Medical_History_Header`, `Modifier`, `O2_Saturation`, `Obesity`, `Oncological`, `Overweight`, `Oxygen_Therapy`, `Pregnancy`, `Procedure`, `Psychological_Condition`, `Pulse`, `Race_Ethnicity`, `Relationship_Status`, `RelativeDate`, `RelativeTime`, `Respiration`, `Route`, `Section_Header`, `Sexually_Active_or_Sexual_Orientation`, `Smoking`, `Social_History_Header`, `Strength`, `Substance`, `Substance_Quantity`, `Symptom`, `Temperature`, `Test`, `Test_Result`, `Time`, `Total_Cholesterol`, `Treatment`, `Triglycerides`, `VS_Finding`, `Vaccine`, `Vaccine_Name`, `Vital_Signs_Header`, `Weight`

- Assertion Status Labels: `Hypothetical`, `Someoneelse`, `Past`, `Absent`, `Family`, `Planned`, `Possible`, `Present`

- Relation Extraction Labels: `is_finding_of`, `is_date_of`, `is_result_of`, `O`


In [ ]:
from sparknlp.pretrained import PretrainedPipeline

In [ ]:
pipeline = PretrainedPipeline("explain_clinical_doc_granular", "en", "clinical/models")

explain_clinical_doc_granular download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [ ]:
pipeline.model.stages

[DocumentAssembler_27dc46af8fb2,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_b17b2f22704f,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_c89cbceb1028,
 NER_CONVERTER_59ce0f445e62,
 NER_CONVERTER_256c8eb7a1be,
 ASSERTION_DL_e5e007602386,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 RelationExtractionModel_ce79d77d1bf1]

In [ ]:
# Load pretrained pipeline from local disk:

# pipeline_local = PretrainedPipeline.from_disk('/root/cache_pretrained/explain_clinical_doc_granular_en_5.2.1_3.4_1706289187844')

#### with fullAnnotate()

In [ ]:
result = pipeline.fullAnnotate("""The patient admitted for gastrointestinal pathology, under working treatment.
History of prior heart murmur with echocardiogram findings as above on March 1998.
According to the latest echocardiogram, basically revealed normal left ventricular function with left atrial enlargement .
Based on the above findings, we will treat her medically with ACE inhibitors and diuretics and see how she fares.""")

In [ ]:
result[0].keys()

dict_keys(['assertion_ner_chunk', 'document', 'jsl_ner_chunk', 'assertion', 'jsl_ner', 'token', 'relations', 'embeddings', 'pos_tags', 'dependencies', 'sentence'])

**NER Results**

In [ ]:
import pandas as pd
chunks=[]
entities=[]
begins=[]
ends=[]

for n in result[0]['jsl_ner_chunk']:

    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities})

df

,chunks,begin,end,entities
0,admitted,12,19,Admission_Discharge
1,gastrointestinal pathology,25,50,Clinical_Dept
2,heart murmur,95,106,Heart_Disease
3,echocardiogram,113,126,Test
4,March 1998,149,158,Date
5,echocardiogram,185,198,Test
6,normal,220,225,Test_Result
7,left ventricular function,227,251,Test
8,left atrial enlargement,258,280,Heart_Disease
9,her,327,329,Gender


**Assertion Status Results**

In [ ]:
chunks=[]
entities=[]
status=[]

for n,m in zip(result[0]['assertion_ner_chunk'],result[0]['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

,chunks,entities,assertion
0,heart murmur,Heart_Disease,Past
1,echocardiogram,Test,Past
2,echocardiogram,Test,Present
3,normal,Test_Result,Present
4,left ventricular function,Test,Present
5,left atrial enlargement,Heart_Disease,Present
6,ACE inhibitors,Drug_Ingredient,Planned
7,diuretics,Drug_Ingredient,Planned


**Relation Extraction Results**

In [ ]:
rel_pairs=[]
for rel in result[0]["relations"]:
    rel_pairs.append((
        rel.result,
        rel.metadata['entity1'],
        rel.metadata['entity1_begin'],
        rel.metadata['entity1_end'],
        rel.metadata['chunk1'],
        rel.metadata['entity2'],
        rel.metadata['entity2_begin'],
        rel.metadata['entity2_end'],
        rel.metadata['chunk2'],
        rel.metadata['confidence']
    ))

rel_df = pd.DataFrame(rel_pairs, columns=['relation',
                                          'entity1',
                                          'entity1_begin',
                                          'entity1_end',
                                          'chunk1',
                                          'entity2',
                                          'entity2_begin',
                                          'entity2_end',
                                          'chunk2',
                                          'confidence'])

rel_df.confidence = rel_df.confidence.astype(float)
rel_df[rel_df.relation!="O"]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,is_finding_of,Heart_Disease,95,106,heart murmur,Test,113,126,echocardiogram,1.0
1,is_date_of,Heart_Disease,95,106,heart murmur,Date,149,158,March 1998,1.0
2,is_date_of,Test,113,126,echocardiogram,Date,149,158,March 1998,1.0
4,is_finding_of,Test,185,198,echocardiogram,Heart_Disease,258,280,left atrial enlargement,1.0
5,is_result_of,Test_Result,220,225,normal,Test,227,251,left ventricular function,1.0
6,is_finding_of,Test,227,251,left ventricular function,Heart_Disease,258,280,left atrial enlargement,1.0


#### with transform()

In [ ]:
data = spark.createDataFrame([["""
The patient admitted for gastrointestinal pathology, under working treatment.
History of prior heart murmur with echocardiogram findings as above on March 1998.
According to the latest echocardiogram, basically revealed normal left ventricular function with left atrial enlargement .
Based on the above findings, we will treat her medically with ACE inhibitors and diuretics and see how she fares.
"""]]).toDF("text")

result = pipeline.transform(data)

result.select(F.explode(F.arrays_zip(result.jsl_ner_chunk.result,
                                     result.jsl_ner_chunk.begin,
                                     result.jsl_ner_chunk.end,
                                     result.jsl_ner_chunk.metadata)).alias("cols"))\
        .select(F.expr("cols['0']").alias("chunk"),
                F.expr("cols['1']").alias("begin"),
                F.expr("cols['2']").alias("end"),
                F.expr("cols['3']['entity']").alias("ner_label")).show()

+--------------------+-----+---+-------------------+
|               chunk|begin|end|          ner_label|
+--------------------+-----+---+-------------------+
|            admitted|   13| 20|Admission_Discharge|
|gastrointestinal ...|   26| 51|      Clinical_Dept|
|        heart murmur|   96|107|      Heart_Disease|
|      echocardiogram|  114|127|               Test|
|          March 1998|  150|159|               Date|
|      echocardiogram|  186|199|               Test|
|              normal|  221|226|        Test_Result|
|left ventricular ...|  228|252|               Test|
|left atrial enlar...|  259|281|      Heart_Disease|
|                 her|  328|330|             Gender|
|      ACE inhibitors|  347|360|    Drug_Ingredient|
|           diuretics|  366|374|    Drug_Ingredient|
|                 she|  388|390|             Gender|
+--------------------+-----+---+-------------------+



In [ ]:
result.select(F.explode(F.arrays_zip(result.assertion_ner_chunk.result,
                                     result.assertion_ner_chunk.begin,
                                     result.assertion_ner_chunk.end,
                                     result.assertion_ner_chunk.metadata,
                                     result.assertion.result)).alias("cols"))\
        .select(F.expr("cols['0']").alias("chunk"),
                F.expr("cols['1']").alias("begin"),
                F.expr("cols['2']").alias("end"),
                F.expr("cols['3']['entity']").alias("ner_label"),
                F.expr("cols['4']").alias("status")).show()

+--------------------+-----+---+---------------+-------+
|               chunk|begin|end|      ner_label| status|
+--------------------+-----+---+---------------+-------+
|        heart murmur|   96|107|  Heart_Disease|   Past|
|      echocardiogram|  114|127|           Test|   Past|
|      echocardiogram|  186|199|           Test|Present|
|              normal|  221|226|    Test_Result|Present|
|left ventricular ...|  228|252|           Test|Present|
|left atrial enlar...|  259|281|  Heart_Disease|Present|
|      ACE inhibitors|  347|360|Drug_Ingredient|Planned|
|           diuretics|  366|374|Drug_Ingredient|Planned|
+--------------------+-----+---+---------------+-------+



In [ ]:
result.select(F.explode(F.arrays_zip(result.token.result,
                                     result.jsl_ner.result,
                                     result.pos_tags.result,
                                     result.dependencies.result)).alias("cols"))\
        .select(F.expr("cols['0']").alias("tokens"),
                F.expr("cols['1']").alias("jsl_ner"),
                F.expr("cols['2']").alias("pos_tags"),
                F.expr("cols['3']").alias("dependencies")).show()

+----------------+--------------------+--------+------------+
|          tokens|             jsl_ner|pos_tags|dependencies|
+----------------+--------------------+--------+------------+
|             The|                   O|      DD|    admitted|
|         patient|                   O|      NN|    admitted|
|        admitted|B-Admission_Disch...|     VVN|        ROOT|
|             for|                   O|      II|   pathology|
|gastrointestinal|     B-Clinical_Dept|      JJ|   pathology|
|       pathology|     I-Clinical_Dept|      NN|    admitted|
|               ,|                   O|      NN|    admitted|
|           under|                   O|      II|   treatment|
|         working|                   O|    VVGJ|   treatment|
|       treatment|                   O|      NN|    admitted|
|               .|                   O|      NN|    admitted|
|         History|                   O|      NN|        ROOT|
|              of|                   O|      II|       heart|
|       

### explain_clinical_doc_carp

A pipeline with ner_clinical, assertion_dl, re_clinical and ner_posology. It will extract clinical and medication entities, assign assertion status and find relationships between clinical entities.

In [ ]:
pipeline = PretrainedPipeline('explain_clinical_doc_carp', 'en', 'clinical/models')

explain_clinical_doc_carp download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [ ]:
pipeline.model.stages

[DocumentAssembler_9619f8fd837c,
 SentenceDetector_c0b14c755033,
 REGEX_TOKENIZER_3087df5b9e9d,
 POS_6f55785005bf,
 dependency_d5a8da6c9093,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_cd5ce67b529f,
 NER_CONVERTER_2f1dcb61b142,
 MedicalNerModel_4a303d875127,
 NER_CONVERTER_a8cff4d56af8,
 ASSERTION_DL_25881ab6309e,
 RelationExtractionModel_9c255241fec3]

In [ ]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus, used to take metformin 1000 mg two times a day, presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .
She was seen by the endocrinology service and discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals.
"""

annotations = pipeline.annotate(text)

annotations.keys()


dict_keys(['sentences', 'clinical_ner_tags', 'document', 'clinical_ner_chunks', 'assertion', 'clinical_relations', 'posology_ner_tags', 'tokens', 'posology_ner_chunks', 'embeddings', 'pos_tags', 'dependencies'])

In [ ]:
import pandas as pd

rows = list(zip(annotations['tokens'], annotations['clinical_ner_tags'], annotations['posology_ner_tags'], annotations['pos_tags'], annotations['dependencies']))

df = pd.DataFrame(rows, columns = ['tokens','clinical_ner_tags','posology_ner_tags','POS_tags','dependencies'])

df.head(20)

,tokens,clinical_ner_tags,posology_ner_tags,POS_tags,dependencies
0,A,O,O,DD,female
1,28-year-old,O,O,NN,female
2,female,O,O,NN,ROOT
3,with,O,O,II,history
4,a,O,O,DD,history
5,history,O,O,NN,female
6,of,O,O,II,history
7,gestational,B-PROBLEM,O,JJ,of
8,diabetes,I-PROBLEM,O,NN,mellitus
9,mellitus,I-PROBLEM,O,NN,gestational


In [ ]:
text = 'Patient has a headache for the last 2 weeks and appears anxious when she walks fast. No alopecia noted. She denies pain'

result = pipeline.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(result['clinical_ner_chunks'],result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

,chunks,entities,assertion
0,a headache,PROBLEM,present
1,anxious,PROBLEM,present
2,alopecia,PROBLEM,absent
3,pain,PROBLEM,absent


In [ ]:
text = """
The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also
given 1 unit of Metformin daily.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night ,
12 units of insulin lispro with meals , and metformin 1000 mg two times a day.
"""

result = pipeline.fullAnnotate(text)[0]

chunks=[]
entities=[]
begins=[]
ends=[]

for n in result['posology_ner_chunks']:

    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities})

df

,chunks,begin,end,entities
0,1 unit,28,33,DOSAGE
1,Advil,38,42,DRUG
2,for 5 days,44,53,DURATION
3,1 unit,95,100,DOSAGE
4,Metformin,105,113,DRUG
5,daily,115,119,FREQUENCY
6,40 units,189,196,DOSAGE
7,insulin glargine,201,216,DRUG
8,at night,218,225,FREQUENCY
9,12 units,229,236,DOSAGE


### explain_clinical_doc_oncology

> This pipeline includes Named-Entity Recognition, Assertion Status and Relation Extraction models to extract information from oncology texts.


- Clinical Entity Labels: `Adenopathy`, `Age`, `Biomarker`,`Biomarker_Result`, `Cancer_Dx`, `Cancer_Score` ,`Cancer_Surgery`, `Chemotherapy`, `Cycle_Count` ,`Cycle_Day`, `Cycle_Number`, `Date` ,`Death_Entity`, `Direction`, `Dosage` ,`Duration`, `Frequency`, `Gender` ,`Grade`, `Histological_Type`, `Hormonal_Therapy` ,`Imaging_Test`, `Immunotherapy`, `Invasion` ,`Line_Of_Therapy`, `Metastasis`, `Oncogene` ,`PROBLEM`, `Pathology_Result`, `Pathology_Test` ,`PROBLEM`, `Performance_Status`, `Race_Ethnicity` ,`Radiotherapy`, `Response_To_Treatment`, `Relative_Date` ,`Route`, `Site_Bone`, `Site_Brain` ,`Site_Breast`, `Site_Liver`, `Site_Lung` ,`Site_Lymph_Node`, `Site_Other_Body_Part`, `Smoking_Status` ,`Staging`, `Targeted_Therapy`, `Tumor_Finding` ,`Tumor_Size`, `Unspecific_Therapy`, `Radiation_Dose` ,`Anatomical_Site`, `Cancer_Therapy`, `Size_Trend` ,`Lymph_Node`, `Tumor_Description`,`Lymph_Node_Modifier`, `Posology_Information`, `Oncological`,`Weight`,`Alcohol`,`Communicable_Disease`,`BMI`,`Obesity`,`Diabetes`

- Assertion Status Labels: `Present`, `Absent`, `Possible`, `Past`, `Family`, `Hypotetical`

- Relation Extraction Labels: `is_size_of`, `is_finding_of`, `is_date_of`, `is_location_of`


In [ ]:
ner_pipeline = PretrainedPipeline("explain_clinical_doc_oncology", "en", "clinical/models")

explain_clinical_doc_oncology download started this may take some time.
Approx size to download 1.8 GB
[OK!]


In [ ]:
ner_pipeline.model.stages

[DocumentAssembler_a70034254885,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_db5325c018e1,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_8c59079bd37d,
 NER_CONVERTER_d44db70d0356,
 MedicalNerModel_f17b93cea1f2,
 NER_CONVERTER_3f1711349b1f,
 MedicalNerModel_58233645f160,
 NER_CONVERTER_67697148287e,
 MedicalNerModel_185d3fbf617b,
 NER_CONVERTER_e5f9606036ad,
 MedicalNerModel_ceff3f64ab20,
 NER_CONVERTER_241608e4a7ee,
 MedicalNerModel_c89cbceb1028,
 NER_CONVERTER_4b4ba0d4371c,
 MedicalNerModel_d674613cb476,
 NER_CONVERTER_4eb12c885cfd,
 MERGE_a905bcc07b50,
 MERGE_33e0102082fa,
 ASSERTION_DL_8d77f383c928,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 RelationExtractionModel_68ebe11369b6,
 RelationExtractionModel_513eb6317779,
 AnnotationMerger_eb205c133eff]

In [ ]:
text ="""The Patient underwent a computed tomography (CT) scan of the abdomen and pelvis,
which showed a complex ovarian mass. A Pap smear performed one month later was positive for
atypical glandular cells suspicious for adenocarcinoma. The pathologic specimen showed extension
of the tumor throughout the fallopian tubes, appendix, omentum, and 5 out of 5 enlarged lymph nodes.
The final pathologic diagnosis of the tumor was stage IIIC papillary serous ovarian adenocarcinoma.
Two months later, the patient was diagnosed with lung metastases.
"""

result = ner_pipeline.fullAnnotate(text)[0]


In [ ]:
result.keys()

dict_keys(['ner_oncology_unspecific_posology_chunk', 're_oncology_granular_wip', 'ner_oncology_anatomy_general_chunk', 'ner_oncology', 'document', 're_oncology_biomarker_result_wip', 'merged_chunk', 'merged_chunk_for_assertion', 'ner_oncology_anatomy_general', 'assertion', 'all_relations', 'ner_posology_chunk', 'ner_oncology_response_to_treatment', 'ner_oncology_chunk', 'token', 'ner_oncology_tnm_chunk', 'ner_jsl_chunk', 'embeddings', 'ner_oncology_tnm', 'pos_tags', 'ner_jsl', 'dependencies', 'ner_oncology_unspecific_posology', 'ner_posology', 'ner_oncology_response_to_treatment_chunk', 'sentence'])

In [ ]:
import pandas as pd

chunks=[]
entities=[]
begins=[]
ends=[]

for n in result['merged_chunk']:

    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities})

df

,chunks,begin,end,entities
0,computed tomography,24,42,Imaging_Test
1,CT,45,46,Imaging_Test
2,abdomen,61,67,Site_Other_Body_Part
3,pelvis,73,78,Site_Other_Body_Part
4,ovarian,105,111,Site_Other_Body_Part
5,mass,113,116,Tumor_Finding
6,Pap smear,121,129,Pathology_Test
7,one month later,141,155,Relative_Date
8,atypical glandular cells,175,198,Pathology_Result
9,adenocarcinoma,215,228,Cancer_Dx


In [ ]:
chunks=[]
entities=[]
status=[]

for n,m in zip(result['merged_chunk_for_assertion'],result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

,chunks,entities,assertion
0,computed tomography,Imaging_Test,Past
1,CT,Imaging_Test,Past
2,mass,Tumor_Finding,Present
3,Pap smear,Pathology_Test,Past
4,atypical glandular cells,Pathology_Result,Present
5,adenocarcinoma,Cancer_Dx,Possible
6,pathologic specimen,Pathology_Test,Past
7,extension,Invasion,Present
8,tumor,Tumor_Finding,Present
9,enlarged,Lymph_Node_Modifier,Present


In [ ]:
import pandas as pd

def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result,
          rel.metadata['entity1'],
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'],
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'],
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  rel_df.confidence = rel_df.confidence.astype(float)

  return rel_df

In [ ]:
annotations = ner_pipeline.fullAnnotate(text)

rel_df = get_relations_df (annotations, 'all_relations')

rel_df[rel_df.relation!="O"]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
2,is_location_of,Site_Other_Body_Part,105,111,ovarian,Tumor_Finding,113,116,mass,0.923225
3,is_finding_of,Pathology_Test,121,129,Pap smear,Cancer_Dx,215,228,adenocarcinoma,0.527566
4,is_location_of,Tumor_Finding,279,283,tumor,Site_Other_Body_Part,300,314,fallopian tubes,0.902630
5,is_location_of,Tumor_Finding,279,283,tumor,Site_Other_Body_Part,317,324,appendix,0.664927


In [ ]:
annotations[0]['all_relations']

[Annotation(category, 61, 116, O, {'chunk2': 'mass', 'confidence': '0.94448596', 'entity2_end': '116', 'chunk1': 'abdomen', 'entity1': 'Site_Other_Body_Part', 'entity2_begin': '113', 'chunk2_confidence': '0.9557', 'entity1_begin': '61', 'sentence': '0', 'direction': 'both', 'entity1_end': '67', 'entity2': 'Tumor_Finding', 'chunk1_confidence': '0.9446'}, []),
 Annotation(category, 73, 116, O, {'chunk2': 'mass', 'confidence': '0.9636435', 'entity2_end': '116', 'chunk1': 'pelvis', 'entity1': 'Site_Other_Body_Part', 'entity2_begin': '113', 'chunk2_confidence': '0.9557', 'entity1_begin': '73', 'sentence': '0', 'direction': 'both', 'entity1_end': '78', 'entity2': 'Tumor_Finding', 'chunk1_confidence': '0.6514'}, []),
 Annotation(category, 105, 116, is_location_of, {'chunk2': 'mass', 'confidence': '0.923225', 'entity2_end': '116', 'chunk1': 'ovarian', 'entity1': 'Site_Other_Body_Part', 'entity2_begin': '113', 'chunk2_confidence': '0.9557', 'entity1_begin': '105', 'sentence': '0', 'direction': 

### explain_clinical_doc_ade

A pipeline for `Adverse Drug Events (ADE)` with `ner_ade_healthcare`, and `classifierdl_ade_biobert`. It will extract `ADE` and `DRUG` clinical entities, and then assign ADE status to a text(`True` means ADE, `False` means not related to ADE). Also extracts relations between `DRUG` and `ADE` entities (`1` means the adverse event and drug entities are related, `0` is not related).

In [ ]:
ade_pipeline = PretrainedPipeline('explain_clinical_doc_ade', 'en', 'clinical/models')

explain_clinical_doc_ade download started this may take some time.
Approx size to download 462.6 MB
[OK!]


In [ ]:
result = ade_pipeline.fullAnnotate("The main adverse effects of Leflunomide consist of diarrhea, nausea, liver enzyme elevation, hypertension, alopecia, and allergic skin reactions.")

result[0].keys()

dict_keys(['bert_sentence_embeddings', 'bert_embeddings', 'document', 'ner_chunks_ade_assertion', 'ner_tags_ade', 'relations_ade_drug', 'ner_chunks_ade', 'assertion_ade', 'tokens', 'class', 'pos_tags', 'dependencies'])

In [ ]:
result[0]['class'][0].metadata

{'sentence': '0', 'False': '0.0033158988', 'True': '0.99668413'}

**NER Results**

In [ ]:
text = """Been taking Lipitor for 15 years , have experienced severe fatigue a lot!!! .
Doctor moved me to voltaren 2 months ago , so far , have only experienced cramps"""

import pandas as pd

chunks = []
entities = []
begin =[]
end = []

print ('sentence:', text)
print()

result = ade_pipeline.fullAnnotate(text)

print ('ADE status:', result[0]['class'][0].result)

print ('prediction probability>> True : ', result[0]['class'][0].metadata['True'], \
        'False: ', result[0]['class'][0].metadata['False'])

for n in result[0]['ner_chunks_ade']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                'begin': begin, 'end': end})

df


sentence: Been taking Lipitor for 15 years , have experienced severe fatigue a lot!!! .
Doctor moved me to voltaren 2 months ago , so far , have only experienced cramps

ADE status: True
prediction probability>> True :  0.9968352 False:  0.0031648325


,chunks,entities,begin,end
0,Lipitor,DRUG,12,18
1,severe fatigue,ADE,52,65
2,voltaren,DRUG,97,104
3,cramps,ADE,152,157


**Assertion Status Results**

In [ ]:
import pandas as pd

text = """Been taking Lipitor for 15 years , have experienced severe fatigue a lot!!! .
Doctor moved me to voltaren 2 months ago , so far , have only experienced cramps"""

print (text)

light_result = ade_pipeline.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ner_chunks_ade_assertion'],light_result['assertion_ade']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

Been taking Lipitor for 15 years , have experienced severe fatigue a lot!!! .
Doctor moved me to voltaren 2 months ago , so far , have only experienced cramps


,chunks,entities,assertion
0,severe fatigue,ADE,conditional
1,cramps,ADE,present


**Relation Extraction Results**

In [ ]:
import pandas as pd

text = """Been taking Lipitor for 15 years , have experienced severe fatigue a lot!!! .
Doctor moved me to voltaren 2 months ago , so far , have only experienced cramps
"""

print (text)

results = ade_pipeline.fullAnnotate(text)

rel_pairs=[]

for rel in results[0]["relations_ade_drug"]:
    rel_pairs.append((
        rel.result,
        rel.metadata['entity1'],
        rel.metadata['entity1_begin'],
        rel.metadata['entity1_end'],
        rel.metadata['chunk1'],
        rel.metadata['entity2'],
        rel.metadata['entity2_begin'],
        rel.metadata['entity2_end'],
        rel.metadata['chunk2'],
        rel.metadata['confidence']
    ))

rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])
rel_df

Been taking Lipitor for 15 years , have experienced severe fatigue a lot!!! .
Doctor moved me to voltaren 2 months ago , so far , have only experienced cramps



,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,1,DRUG,12,18,Lipitor,ADE,52,65,severe fatigue,1.0
1,1,DRUG,12,18,Lipitor,ADE,152,157,cramps,1.0
2,0,ADE,52,65,severe fatigue,DRUG,97,104,voltaren,0.5006272
3,1,DRUG,97,104,voltaren,ADE,152,157,cramps,0.99999857


### exlain_clinical_doc_medication

> A pipeline for detecting posology entities with the `ner_posology_large` NER model, assigning their assertion status with `assertion_jsl` model, and extracting relations between posology-related terminology with `posology_re` relation extraction model.

In [ ]:
medication_pipeline = PretrainedPipeline('explain_clinical_doc_medication', 'en', 'clinical/models')

explain_clinical_doc_medication download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [ ]:
medication_pipeline.model.stages

[DocumentAssembler_5b6b25ae3a32,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_3aaa9b415841,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_5e6f59103f25,
 NER_CONVERTER_c6fab90a28d5,
 NER_CONVERTER_3b6a5b14db49,
 ASSERTION_DL_e5e007602386,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 PosologyREModel_d7fe3a9e8310]

In [ ]:
text = """The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2. She received a course of Bactrim for 14 days for UTI.
She was prescribed 5000 units of Fragmin  subcutaneously daily, and along with Lantus 40 units subcutaneously at bedtime."""

result = medication_pipeline.fullAnnotate(text)[0]

In [ ]:
result.keys()

dict_keys(['assertion_ner_chunk', 'document', 'assertion', 'ner_posology_chunk', 'token', 'relations', 'embeddings_clinical', 'pos_tags', 'dependencies', 'ner_posology', 'sentence'])

In [ ]:
chunks=[]
entities=[]
begins=[]
ends=[]

for n in result['ner_posology_chunk']:

    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities})

df

,chunks,begin,end,entities
0,insulin,59,65,DRUG
1,Bactrim,120,126,DRUG
2,for 14 days,128,138,DURATION
3,5000 units,168,177,DOSAGE
4,Fragmin,182,188,DRUG
5,subcutaneously,191,204,ROUTE
6,daily,206,210,FREQUENCY
7,Lantus,228,233,DRUG
8,40 units,235,242,DOSAGE
9,subcutaneously,244,257,ROUTE


In [ ]:
chunks=[]
entities=[]
status=[]

for n,m in zip(result['assertion_ner_chunk'],result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

,chunks,entities,assertion
0,insulin,DRUG,Family
1,Bactrim,DRUG,Past
2,Fragmin,DRUG,Planned
3,Lantus,DRUG,Past


In [ ]:
annotations = medication_pipeline.fullAnnotate(text)

rel_df = get_relations_df(annotations, 'relations')

rel_df

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,DRUG-DURATION,DRUG,120,126,Bactrim,DURATION,128,138,for 14 days,1.0
1,DOSAGE-DRUG,DOSAGE,168,177,5000 units,DRUG,182,188,Fragmin,1.0
2,DRUG-ROUTE,DRUG,182,188,Fragmin,ROUTE,191,204,subcutaneously,1.0
3,DRUG-FREQUENCY,DRUG,182,188,Fragmin,FREQUENCY,206,210,daily,1.0
4,DRUG-DOSAGE,DRUG,228,233,Lantus,DOSAGE,235,242,40 units,1.0
5,DRUG-ROUTE,DRUG,228,233,Lantus,ROUTE,244,257,subcutaneously,1.0
6,DRUG-FREQUENCY,DRUG,228,233,Lantus,FREQUENCY,259,268,at bedtime,1.0


In [ ]:
annotations[0]['relations']

[Annotation(category, 120, 138, DRUG-DURATION, {'chunk2': 'for 14 days', 'confidence': '1.0', 'entity2_end': '138', 'chunk1': 'Bactrim', 'entity1': 'DRUG', 'entity2_begin': '128', 'chunk2_confidence': '0.79349995', 'entity1_begin': '120', 'sentence': '1', 'direction': 'both', 'entity1_end': '126', 'entity2': 'DURATION', 'chunk1_confidence': '0.9994'}, []),
 Annotation(category, 168, 188, DOSAGE-DRUG, {'chunk2': 'Fragmin', 'confidence': '1.0', 'entity2_end': '188', 'chunk1': '5000 units', 'entity1': 'DOSAGE', 'entity2_begin': '182', 'chunk2_confidence': '0.9996', 'entity1_begin': '168', 'sentence': '2', 'direction': 'both', 'entity1_end': '177', 'entity2': 'DRUG', 'chunk1_confidence': '0.80009997'}, []),
 Annotation(category, 182, 204, DRUG-ROUTE, {'chunk2': 'subcutaneously', 'confidence': '1.0', 'entity2_end': '204', 'chunk1': 'Fragmin', 'entity1': 'DRUG', 'entity2_begin': '191', 'chunk2_confidence': '0.9994', 'entity1_begin': '182', 'sentence': '2', 'direction': 'both', 'entity1_end':

### explain_clinical_doc_radiology

> A pipeline for detecting radiology entities with the `ner_radiology` NER model, assigning their assertion status with `assertion_dl_radiology` model, and extracting relations between the diagnosis, test, and findings with `re_test_problem_finding` relation extraction model.

In [ ]:
radiology_pipeline = PretrainedPipeline('explain_clinical_doc_radiology', 'en', 'clinical/models')

explain_clinical_doc_radiology download started this may take some time.
Approx size to download 1.7 GB
[OK!]


In [ ]:
radiology_pipeline.model.stages

[DocumentAssembler_7be744fefcc1,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_a430ec88107f,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_f7f58f2addf7,
 NER_CONVERTER_054a7661a221,
 MedicalNerModel_32430beeafc0,
 NER_CONVERTER_c95e9906feb9,
 MedicalNerModel_5e6f59103f25,
 NER_CONVERTER_6555d615be71,
 MedicalNerModel_8c59079bd37d,
 NER_CONVERTER_fdf91325f85c,
 MedicalNerModel_c89cbceb1028,
 NER_CONVERTER_404f44a63e5b,
 MERGE_b2f5b8021e11,
 MERGE_c314756a9d4c,
 ASSERTION_DL_614cf4bf71de,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 RelationExtractionModel_853993778cd5]

In [ ]:
text = """Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder.
This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow.
This may represent benign fibrous tissue or a lipoma."""

result = radiology_pipeline.fullAnnotate(text)[0]

In [ ]:
result.keys()

dict_keys(['radiology_ner_chunk', 'ner_chexpert_chunk', 'posology_ner', 'ner_oncology', 'document', 'ner_chexpert', 'merged_chunk', 'posology_ner_chunk', 'merged_chunk_for_assertion', 'jsl_ner_chunk', 'radiology_ner', 'assertion', 'jsl_ner', 'ner_oncology_chunk', 'token', 'relations', 'embeddings', 'pos_tags', 'dependencies', 'sentence'])

In [ ]:
chunks=[]
entities=[]
begins=[]
ends=[]

for n in result['radiology_ner_chunk']:

    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities})

df

,chunks,begin,end,entities
0,Bilateral breast,0,15,BodyPart
1,ultrasound,17,26,Imaging_Test
2,ovoid mass,78,87,ImagingFindings
3,0.5 x 0.5 x 0.4,113,127,Measurements
4,cm,129,130,Units
5,anteromedial aspect of the left shoulder,163,202,BodyPart
6,mass,210,213,ImagingFindings
7,isoechoic echotexture,228,248,ImagingFindings
8,muscle,266,271,BodyPart
9,internal color flow,294,312,ImagingFindings


In [ ]:
chunks=[]
entities=[]
status=[]

for n,m in zip(result['merged_chunk_for_assertion'],result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

,chunks,entities,assertion
0,ovoid mass,ImagingFindings,Confirmed
1,mass,ImagingFindings,Confirmed
2,isoechoic echotexture,ImagingFindings,Confirmed
3,internal color flow,ImagingFindings,Negative
4,benign fibrous tissue,ImagingFindings,Suspected
5,lipoma,Disease_Syndrome_Disorder,Suspected


In [ ]:
annotations = radiology_pipeline.fullAnnotate(text)

rel_df = get_relations_df(annotations, 'relations')

rel_df

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,is_related,BodyPart,0,15,Bilateral breast,Imaging_Test,17,26,ultrasound,1.000000
1,is_related,BodyPart,0,15,Bilateral breast,ImagingFindings,78,87,ovoid mass,0.999997
2,is_related,Imaging_Test,17,26,ultrasound,ImagingFindings,78,87,ovoid mass,0.999569
3,is_related,ImagingFindings,78,87,ovoid mass,Measurements,113,130,0.5 x 0.5 x 0.4 cm,1.000000
4,is_related,ImagingFindings,210,213,mass,BodyPart,257,271,adjacent muscle,0.997639
5,is_related,ImagingFindings,228,248,isoechoic echotexture,BodyPart,257,271,adjacent muscle,0.999999


In [ ]:
annotations[0]['relations']

[Annotation(category, 0, 26, is_related, {'chunk2': 'ultrasound', 'confidence': '1.0', 'entity2_end': '26', 'chunk1': 'Bilateral breast', 'entity1': 'BodyPart', 'entity2_begin': '17', 'chunk2_confidence': '0.6734', 'entity1_begin': '0', 'sentence': '0', 'direction': 'both', 'entity1_end': '15', 'entity2': 'Imaging_Test', 'chunk1_confidence': '0.945'}, []),
 Annotation(category, 0, 87, is_related, {'chunk2': 'ovoid mass', 'confidence': '0.99999714', 'entity2_end': '87', 'chunk1': 'Bilateral breast', 'entity1': 'BodyPart', 'entity2_begin': '78', 'chunk2_confidence': '0.6095', 'entity1_begin': '0', 'sentence': '0', 'direction': 'both', 'entity1_end': '15', 'entity2': 'ImagingFindings', 'chunk1_confidence': '0.945'}, []),
 Annotation(category, 17, 87, is_related, {'chunk2': 'ovoid mass', 'confidence': '0.99956936', 'entity2_end': '87', 'chunk1': 'ultrasound', 'entity1': 'Imaging_Test', 'entity2_begin': '78', 'chunk2_confidence': '0.6095', 'entity1_begin': '17', 'sentence': '0', 'direction'

### Clinical Deidentification

This pipeline can be used to deidentify PHI information from medical texts. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate `AGE`, `CONTACT`, `DATE`, `ID`, `LOCATION`, `NAME`, `PROFESSION`, `CITY`, `COUNTRY`, `DOCTOR`, `HOSPITAL`, `IDNUM`, `MEDICALRECORD`, `ORGANIZATION`, `PATIENT`, `PHONE`, `PROFESSION`, `STREET`, `USERNAME`, `ZIP`, `ACCOUNT`, `LICENSE`, `VIN`, `SSN`, `DLN`, `PLATE`, `IPADDR` entities.

|index|model|lang|
|-----:|:-----|----|
| 1| [clinical_deidentification](https://nlp.johnsnowlabs.com/2022/03/03/clinical_deidentification_de_3_0.html)  |de|
| 2| [clinical_deidentification](https://nlp.johnsnowlabs.com/2021/05/27/clinical_deidentification_en.html)  |en|
| 3| [clinical_deidentification_glove](https://nlp.johnsnowlabs.com/2022/03/04/clinical_deidentification_glove_en_3_0.html)  |en|
| 4| [clinical_deidentification_glove_augmented](https://nlp.johnsnowlabs.com/2022/03/22/clinical_deidentification_glove_augmented_en_3_0.html)  |en|
| 5| [clinical_deidentification](https://nlp.johnsnowlabs.com/2022/03/02/clinical_deidentification_es_2_4.html)  |es|
| 6| [clinical_deidentification_augmented](https://nlp.johnsnowlabs.com/2022/03/03/clinical_deidentification_augmented_es_2_4.html)  |es|
| 7| [clinical_deidentification](https://nlp.johnsnowlabs.com/2022/03/04/clinical_deidentification_fr_2_4.html)  |fr|
| 8| [clinical_deidentification](https://nlp.johnsnowlabs.com/2022/03/28/clinical_deidentification_it_3_0.html)  |it|
| 9| [clinical_deidentification](https://nlp.johnsnowlabs.com/2022/06/21/clinical_deidentification_pt_3_0.html)  |pt|
| 10| [clinical_deidentification](https://nlp.johnsnowlabs.com/2022/06/28/clinical_deidentification_ro_3_0.html)  |ro|
| 11| [clinical_deidentification](https://nlp.johnsnowlabs.com/2023/06/22/clinical_deidentification_ar.html)  |ar|


You can find **`German`, `Spanish`, `French`, `Italian`, `Portuguese`, `Romanian`**  and **`Arabic`**  deidentification models and pretrained pipeline examples in this notebook:   [Clinical Multi Language Deidentification Notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.1.Clinical_Multi_Language_Deidentification.ipynb)


In [ ]:
deid_pipeline = PretrainedPipeline("clinical_deidentification", "en", "clinical/models")

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [ ]:
deid_res = deid_pipeline.annotate("Record date : 2093-01-13 , David Hale , M.D .  Name : Hendrickson , Ora MR 25 years-old . # 719435 Date : 01/13/93 . Signed by Oliveira Sander . Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street. Phone 302-786-5227.")

In [ ]:
deid_res.keys()

dict_keys(['masked', 'obfuscated', 'ner_chunk', 'masked_fixed_length_chars', 'sentence', 'masked_with_chars'])

In [ ]:
pd.set_option("display.max_colwidth", 100)

df= pd.DataFrame(list(zip(deid_res["sentence"],
                          deid_res["masked"],
                          deid_res["masked_with_chars"],
                          deid_res["masked_fixed_length_chars"],
                          deid_res["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <DOCTOR> , M.D .","Record date : [********] , [********] , M.D .","Record date : **** , **** , M.D .","Record date : 2093-01-25 , Remi Deter , M.D ."
1,"Name : Hendrickson , Ora MR 25 years-old .",Name : <PATIENT> MR <AGE> years-old .,Name : [***************] MR ** years-old .,Name : **** MR **** years-old .,Name : Lynne Logan MR 39 years-old .
2,# 719435 Date : 01/13/93 .,# <MEDICALRECORD> Date : <DATE> .,# [****] Date : [******] .,# **** Date : **** .,# 161096 Date : 01/25/93 .
3,Signed by Oliveira Sander .,Signed by <DOCTOR> .,Signed by [*************] .,Signed by **** .,Signed by Jan Fireman .
4,Record date : 2079-11-09 .,Record date : <DATE> .,Record date : [********] .,Record date : **** .,Record date : 2079-11-21 .
5,Cocke County Baptist Hospital . 0295 Keats Street.,<LOCATION>.,[***********************************************].,****.,"261 Mack Avenue,7Th Floor."
6,Phone 302-786-5227.,Phone <PHONE>.,Phone [**********].,Phone ****.,Phone 045-409-8119.


## NER Pipelines

**`NER pretrained ` Model List**

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [jsl_ner_wip_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_ner_wip_clinical_pipeline_en_3_0.html)  | 2| [jsl_ner_wip_greedy_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_ner_wip_greedy_biobert_pipeline_en_3_0.html)  | 3| [jsl_ner_wip_greedy_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_ner_wip_greedy_clinical_pipeline_en_3_0.html)  | 4| [jsl_ner_wip_modifier_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_ner_wip_modifier_clinical_pipeline_en_3_0.html)  |
| 5| [jsl_rd_ner_wip_greedy_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_rd_ner_wip_greedy_biobert_pipeline_en_3_0.html)  | 6| [jsl_rd_ner_wip_greedy_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/jsl_rd_ner_wip_greedy_clinical_pipeline_en_3_0.html)  | 7| [ner_abbreviation_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_abbreviation_clinical_pipeline_en_3_0.html)  | 8| [ner_ade_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_ade_biobert_pipeline_en_3_0.html)  |
| 9| [ner_ade_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_ade_clinical_pipeline_en_3_0.html)  | 10| [ner_ade_clinicalbert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_ade_clinicalbert_pipeline_en_3_0.html)  | 11| [ner_ade_healthcare_pipeline](https://nlp.johnsnowlabs.com/2022/03/22/ner_ade_healthcare_pipeline_en_3_0.html)  | 12| [ner_anatomy_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_anatomy_biobert_pipeline_en_3_0.html)  |
| 13| [ner_anatomy_coarse_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_anatomy_coarse_biobert_pipeline_en_3_0.html)  | 14| [ner_anatomy_coarse_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_anatomy_coarse_pipeline_en_3_0.html)  | 15| [ner_anatomy_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_anatomy_pipeline_en_3_0.html)  | 16| [ner_bacterial_species_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_bacterial_species_pipeline_en_3_0.html)  |
| 17| [ner_biomarker_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_biomarker_pipeline_en_3_0.html)  | 18| [ner_biomedical_bc2gm_pipeline](https://nlp.johnsnowlabs.com/2022/06/22/ner_biomedical_bc2gm_pipeline_en_3_0.html)  | 19| [ner_bionlp_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_bionlp_biobert_pipeline_en_3_0.html)  | 20| [ner_bionlp_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_bionlp_pipeline_en_3_0.html)  |
| 21| [ner_cancer_genetics_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_cancer_genetics_pipeline_en_3_0.html)  | 22| [ner_cellular_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_cellular_biobert_pipeline_en_3_0.html)  | 23| [ner_cellular_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_cellular_pipeline_en_3_0.html)  | 24| [ner_chemicals_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_chemicals_pipeline_en_3_0.html)  |
| 25| [ner_chemprot_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_chemprot_biobert_pipeline_en_3_0.html)  | 26| [ner_chemprot_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_chemprot_clinical_pipeline_en_3_0.html)  | 27| [ner_chexpert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_chexpert_pipeline_en_3_0.html)  | 28| [ner_clinical_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_clinical_biobert_pipeline_en_3_0.html)  |
| 29| [ner_clinical_large_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_clinical_large_pipeline_en_3_0.html)  | 30| [ner_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_clinical_pipeline_en_3_0.html)  | 31| [ner_clinical_trials_abstracts_pipeline](https://nlp.johnsnowlabs.com/2022/06/27/ner_clinical_trials_abstracts_pipeline_en_3_0.html)  | 32| [ner_diseases_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_diseases_biobert_pipeline_en_3_0.html)  |
| 33| [ner_diseases_large_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_diseases_large_pipeline_en_3_0.html)  | 34| [ner_diseases_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_diseases_pipeline_en_3_0.html)  | 35| [ner_drugprot_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_drugprot_clinical_pipeline_en_3_0.html)  | 36| [ner_drugs_greedy_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_drugs_greedy_pipeline_en_3_0.html)  |
| 37| [ner_drugs_large_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_drugs_large_pipeline_en_3_0.html)  | 38| [ner_drugs_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_drugs_pipeline_en_3_0.html)  | 39| [ner_events_admission_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_events_admission_clinical_pipeline_en_3_0.html)  | 40| [ner_events_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_events_biobert_pipeline_en_3_0.html)  |
| 41| [ner_events_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_events_clinical_pipeline_en_3_0.html)  | 42| [ner_events_healthcare_pipeline](https://nlp.johnsnowlabs.com/2022/03/22/ner_events_healthcare_pipeline_en_3_0.html)  | 43| [ner_genetic_variants_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_genetic_variants_pipeline_en_3_0.html)  | 44| [ner_healthcare_pipeline](https://nlp.johnsnowlabs.com/2022/03/22/ner_healthcare_pipeline_en_3_0.html)  |
| 45| [ner_human_phenotype_gene_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_human_phenotype_gene_biobert_pipeline_en_3_0.html)  | 46| [ner_human_phenotype_gene_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_human_phenotype_gene_clinical_pipeline_en_3_0.html)  | 47| [ner_human_phenotype_go_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_human_phenotype_go_biobert_pipeline_en_3_0.html)  | 48| [ner_human_phenotype_go_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_human_phenotype_go_clinical_pipeline_en_3_0.html)  |
| 49| [ner_jsl_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_biobert_pipeline_en_3_0.html)  | 50| [ner_jsl_enriched_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_enriched_biobert_pipeline_en_3_0.html)  | 51| [ner_jsl_enriched_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_enriched_pipeline_en_3_0.html)  | 52| [ner_jsl_greedy_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_greedy_biobert_pipeline_en_3_0.html)  |
| 53| [ner_jsl_greedy_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_greedy_pipeline_en_3_0.html)  | 54| [ner_jsl_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_pipeline_en_3_0.html)  | 55| [ner_jsl_slim_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_jsl_slim_pipeline_en_3_0.html)  | 56| [ner_measurements_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_measurements_clinical_pipeline_en_3_0.html)  |
| 57| [ner_medmentions_coarse_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_medmentions_coarse_pipeline_en_3_0.html)  | 58| [ner_nihss_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_nihss_pipeline_en_3_0.html)  | 59| [ner_pathogen_pipeline](https://nlp.johnsnowlabs.com/2022/06/29/ner_pathogen_pipeline_en_3_0.html)  | 60| [ner_posology_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_biobert_pipeline_en_3_0.html)  |
| 61| [ner_posology_experimental_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_experimental_pipeline_en_3_0.html)  | 62| [ner_posology_greedy_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_greedy_pipeline_en_3_0.html)  | 63| [ner_posology_healthcare_pipeline](https://nlp.johnsnowlabs.com/2022/03/22/ner_posology_healthcare_pipeline_en_3_0.html)  | 64| [ner_posology_large_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_large_biobert_pipeline_en_3_0.html)  |
| 65| [ner_posology_large_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_large_pipeline_en_3_0.html)  | 66| [ner_posology_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_pipeline_en_3_0.html)  | 67| [ner_posology_small_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_posology_small_pipeline_en_3_0.html)  | 68| [ner_radiology_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_radiology_pipeline_en_3_0.html)  |
| 69| [ner_radiology_wip_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_radiology_wip_clinical_pipeline_en_3_0.html)  | 70| [ner_risk_factors_biobert_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_risk_factors_biobert_pipeline_en_3_0.html)  | 71| [ner_risk_factors_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/ner_risk_factors_pipeline_en_3_0.html)  | 72| [ner_medication_pipeline](https://nlp.johnsnowlabs.com/2022/07/26/ner_medication_pipeline_en_3_0.html)|


**Let's show an example of `ner_jsl_pipeline` can label clinical entities with about 80 different labels.**

In [ ]:
ner_pipeline = PretrainedPipeline('ner_jsl_pipeline', 'en', 'clinical/models')

ner_jsl_pipeline download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [ ]:
ner_pipeline.model.stages

[DocumentAssembler_cefadf0e0f93,
 SentenceDetectorDLModel_c83c27f46b97,
 REGEX_TOKENIZER_cef816a12166,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_c89cbceb1028,
 NER_CONVERTER_42a801d9e143]

In [ ]:
text = """A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .
Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity .
Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia .
The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L .
The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours .
Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day .
It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge ."""

greedy_result = ner_pipeline.fullAnnotate(text)[0]

In [ ]:
greedy_result.keys()

dict_keys(['document', 'ner_chunk', 'token', 'ner', 'embeddings', 'sentence'])

In [ ]:
import pandas as pd

chunks=[]
entities=[]
begins=[]
ends=[]

for n in greedy_result['ner_chunk']:

    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities})

df

,chunks,begin,end,entities
0,28-year-old,2,12,Age
1,female,14,19,Gender
2,gestational diabetes mellitus,39,67,Diabetes
3,eight years prior,79,95,RelativeDate
4,type two diabetes mellitus,128,153,Diabetes
...,...,...,...,...
116,two times a day,2357,2371,Frequency
117,SGLT2 inhibitors,2402,2417,Drug_Ingredient
118,She,2457,2459,Gender
119,endocrinology,2486,2498,Clinical_Dept


## Bert Based NER Pipelines


**`bert token classification pretrained ` Model List**

| Index | Model                                                                                       | Index | Model                                                                                      |
| ----  | ------------------------------------------------------------------------------------------- | ----  | ------------------------------------------------------------------------------------------ |
| 1.    | [bert_token_classifier_ade_tweet_binary_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ade_tweet_binary_pipeline_en.html)                                      | 19.   | [bert_token_classifier_ner_chemicals_pipeline](https://nlp.johnsnowlabs.com/2022/03/14/bert_token_classifier_ner_chemicals_pipeline_en_2_4.html)                                        |
| 2.    | [bert_token_classifier_disease_mentions_tweet_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_disease_mentions_tweet_pipeline_es.html)                                | 20.   | [bert_token_classifier_ner_chemprot_pipeline](https://nlp.johnsnowlabs.com/2022/03/15/bert_token_classifier_ner_chemprot_pipeline_en_2_4.html)                                         |
| 3.    | [bert_token_classifier_ner_jsl](https://nlp.johnsnowlabs.com/2021/08/28/bert_token_classifier_ner_jsl_en.html)                                                          | 21.   | [bert_token_classifier_ner_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/15/bert_token_classifier_ner_clinical_pipeline_en_2_4.html)                                         |
| 4.    | [bert_token_classifier_drug_development_trials_pipeline](https://nlp.johnsnowlabs.com/2022/03/23/bert_token_classifier_drug_development_trials_pipeline_en_3_0.html)                                | 22.   | [bert_token_classifier_ner_clinical_trials_abstracts_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_clinical_trials_abstracts_pipeline_en.html)                                          |
| 5.    | [bert_token_classifier_dutch_udlassy_ner_pipeline](https://nlp.johnsnowlabs.com/2022/04/19/bert_token_classifier_dutch_udlassy_ner_pipeline_nl_3_0.html)                                      | 23.   | [bert_token_classifier_ner_deid_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_deid_pipeline_en_3_0.html)                                             |
| 6.    | [bert_token_classifier_hi_en_ner_pipeline](https://nlp.johnsnowlabs.com/2022/03/22/bert_token_classifier_hi_en_ner_pipeline_hi_3_0.html)                                              | 24.   | [bert_token_classifier_drug_development_trials](https://nlp.johnsnowlabs.com/2021/12/17/bert_token_classifier_drug_development_trials_en.html)                                       |
| 7.    | [bert_token_classifier_negation_uncertainty_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_negation_uncertainty_pipeline_es.html)                       | 25.   | [bert_token_classifier_ner_drugs_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_drugs_pipeline_en_3_0.html)                                            |
| 8.    | [bert_token_classifier_ner_ade_binary_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_ade_binary_pipeline_en.html)                                  | 26.   | [bert_token_classifier_ner_jnlpba_cellular_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_jnlpba_cellular_pipeline_en.html)                                  |
| 9.    | [bert_token_classifier_ner_ade_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_ade_pipeline_en_3_0.html)                                           | 27.   | [bert_token_classifier_ner_jsl_pipeline](https://nlp.johnsnowlabs.com/2022/03/23/bert_token_classifier_ner_jsl_pipeline_en_3_0.html)                                              |
| 10.   | [bert_token_classifier_ner_anatem_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_anatem_pipeline_en.html)                                      | 28.   | [bert_token_classifier_ner_jsl_slim_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_jsl_slim_pipeline_en_3_0.html)                                 |
| 11.   | [bert_token_classifier_ner_anatomy_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_anatomy_pipeline_en_3_0.html)                                 | 29.   | [bert_token_classifier_ner_linnaeus_species_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_linnaeus_species_pipeline_en.html)                          |
| 12.   | [bert_token_classifier_ner_bacteria_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_bacteria_pipeline_en_3_0.html)                               | 30.   | [bert_token_classifier_ner_living_species_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_living_species_pipeline_en.html)                      |
| 13.   | [bert_token_classifier_ner_bc2gm_gene_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_bc2gm_gene_pipeline_en.html)                                   | 31.   | [bert_token_classifier_ner_ncbi_disease_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_ncbi_disease_pipeline_en.html)                            |
| 14.   | [bert_token_classifier_ner_bc4chemd_chemicals_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_bc4chemd_chemicals_pipeline_en.html)                       | 32.   | [bert_token_classifier_ner_pathogen_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_pathogen_pipeline_en.html)                                     |
| 15.   | [bert_token_classifier_ner_bc5cdr_chemicals_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_bc5cdr_chemicals_pipeline_en.html)                       | 33.   | [bert_token_classifier_ner_species_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_species_pipeline_en.html)                                      |
| 16.   | [bert_token_classifier_ner_bc5cdr_disease_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_ner_bc5cdr_disease_pipeline_en.html)                       | 34.   | [bert_token_classifier_pharmacology_pipeline](https://nlp.johnsnowlabs.com/2023/03/20/bert_token_classifier_pharmacology_pipeline_es.html)                                 |
| 17.   | [bert_token_classifier_ner_bionlp_pipeline](https://nlp.johnsnowlabs.com/2022/03/21/bert_token_classifier_ner_bionlp_pipeline_en_3_0.html)                               | 35.   | [bert_token_classifier_scandi_ner_pipeline](https://nlp.johnsnowlabs.com/2022/02/15/bert_token_classifier_scandi_ner_pipeline_xx.html)                                    |
| 18.   | [bert_token_classifier_ner_cellular_pipeline](https://nlp.johnsnowlabs.com/2022/03/09/bert_token_classifier_ner_cellular_pipeline_en_3_0.html)                               |                                                                                                                      |


**Let's show an example of `bert_token_classifier_ner_drugs_pipeline` can extract `DRUG` entities in clinical texts.**

In [ ]:
bert_token_pipeline = PretrainedPipeline("bert_token_classifier_ner_drugs_pipeline", "en", "clinical/models")

bert_token_classifier_ner_drugs_pipeline download started this may take some time.
Approx size to download 386.1 MB
[OK!]


In [ ]:
bert_token_pipeline.model.stages

[DocumentAssembler_fbb1736f8270,
 SentenceDetectorDLModel_8aaebf7e098e,
 REGEX_TOKENIZER_bd5df3943e2f,
 BERT_FOR_TOKEN_CLASSIFICATION_3fa6213c0542,
 NER_CONVERTER_70b935c1d6d8]

In [ ]:
test_sentence = """The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population. The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle. The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.BACKGROUND: At present, it is one of the most important issues for the treatment of breast cancer to develop the standard therapy for patients previously treated with anthracyclines and taxanes. With the objective of determining the usefulnessof vinorelbine monotherapy in patients with advanced or recurrent breast cancerafter standard therapy, we evaluated the efficacy and safety of vinorelbine inpatients previously treated with anthracyclines and taxanes."""

bert_result = bert_token_pipeline.fullAnnotate(test_sentence)[0]

In [ ]:
bert_result.keys()

dict_keys(['document', 'ner_chunk', 'token', 'ner', 'sentence'])

In [ ]:
bert_result["ner_chunk"][0]

Annotation(chunk, 92, 100, potassium, {'chunk': '0', 'confidence': '0.9902544', 'ner_source': 'ner_chunk', 'entity': 'DrugChem', 'sentence': '0'}, [])

In [ ]:
import pandas as pd

chunks=[]
entities=[]
begins=[]
ends=[]

for n in bert_result['ner_chunk']:

    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity'])

df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities})

df

,chunks,begin,end,entities
0,potassium,92,100,DrugChem
1,nucleotide,471,480,DrugChem
2,anthracyclines,1124,1137,DrugChem
3,taxanes,1143,1149,DrugChem
4,vinorelbine,1203,1213,DrugChem
5,vinorelbine,1343,1353,DrugChem
6,anthracyclines,1390,1403,DrugChem
7,taxanes,1409,1415,DrugChem


## NER Profiling Pipelines

We can use pretrained NER profiling pipelines for exploring all the available pretrained NER models at once. In Spark NLP we have two different NER profiling pipelines;

- `ner_profiling_clinical` : Returns results for clinical NER models trained with `embeddings_clinical`.


|| | | |
|--------------|-----------------|-----------------|-----------------|
| jsl_ner_wip_clinical | jsl_ner_wip_greedy_clinical | jsl_ner_wip_modifier_clinical | jsl_rd_ner_wip_greedy_clinical |
| ner_abbreviation_clinical | ner_ade_binary | ner_ade_clinical | ner_anatomy |
| ner_anatomy_coarse | ner_bacterial_species | ner_biomarker | ner_biomedical_bc2gm |
| ner_bionlp | ner_cancer_genetics | ner_cellular | ner_chemd_clinical |
| ner_chemicals | ner_chemprot_clinical | ner_chexpert | ner_clinical |
| ner_clinical_large | ner_clinical_trials_abstracts | ner_covid_trials | ner_deid_augmented |
| ner_deid_enriched | ner_deid_generic_augmented | ner_deid_large | ner_deid_sd |
| ner_deid_sd_large | ner_deid_subentity_augmented | ner_deid_subentity_augmented_i2b2 | ner_deid_synthetic |
| ner_deidentify_dl | ner_diseases | ner_diseases_large | ner_drugprot_clinical |
| ner_drugs | ner_drugs_greedy | ner_drugs_large | ner_events_admission_clinical |
| ner_events_clinical | ner_genetic_variants | ner_human_phenotype_gene_clinical | ner_human_phenotype_go_clinical |
| ner_jsl | ner_jsl_enriched | ner_jsl_greedy | ner_jsl_slim |
| ner_living_species | ner_measurements_clinical | ner_medmentions_coarse | ner_nature_nero_clinical |
| ner_nihss | ner_pathogen | ner_posology | ner_posology_experimental |
| ner_posology_greedy | ner_posology_large | ner_posology_small | ner_radiology |
| ner_radiology_wip_clinical | ner_risk_factors | ner_supplement_clinical | nerdl_tumour_demo |




- `ner_profiling_biobert` : Returns results for clinical NER models trained with `biobert_pubmed_base_cased`.

| | |
|-|-|
| ner_cellular_biobert           | ner_clinical_biobert             |
| ner_diseases_biobert           | ner_anatomy_coarse_biobert       |
| ner_events_biobert             | ner_human_phenotype_gene_biobert |
| ner_bionlp_biobert             | ner_posology_large_biobert       |
| ner_jsl_greedy_biobert         | jsl_rd_ner_wip_greedy_biobert    |
| ner_jsl_biobert                | ner_posology_biobert             |
| ner_anatomy_biobert            | jsl_ner_wip_greedy_biobert       |
| ner_jsl_enriched_biobert       | ner_chemprot_biobert             |
| ner_human_phenotype_go_biobert | ner_ade_biobert                  |
| ner_deid_biobert               | ner_risk_factors_biobert         |
| ner_deid_enriched_biobert      | ner_living_species_biobert                                |


- `ner_profiling_oncology` :This pipeline can be used to explore all the available pretrained NER models at once for Oncology. When you run this pipeline over your text, you will end up with the predictions coming out of each pretrained clinical NER model trained with embeddings_clinical


|                 |                 |                 |                 |
| --------------- | --------------- | --------------- | --------------- |
| 1. ner_oncology_unspecific_posology      | 2. ner_oncology_tnm                      | 3. ner_oncology_therapy                  | 4. ner_oncology_test                     |
| 5. ner_oncology_response_to_treatment    | 6. ner_oncology_posology                 | 7. ner_oncology                          | 8. ner_oncology_limited_80p_for_benchmarks|
| 9. ner_oncology_diagnosis                | 10. ner_oncology_demographics             | 11. ner_oncology_biomarker                | 12. ner_oncology_anatomy_granular         |
| 13. ner_oncology_anatomy_general          |                 |                 |                 |


- `ner_profiling_vop` : This pipeline can be used to explore all the available pretrained NER models at once for Voice of Patients. When you run this pipeline over your text, you will end up with the predictions coming out of each pretrained clinical NER model trained with embeddings_clinical.


|                 |                 |                 |                 |
| --------------- | --------------- | --------------- | --------------- |
| 1. ner_oncology_unspecific_posology         | 2. ner_oncology_tnm                       | 3. ner_oncology_therapy                   | 4. ner_oncology_test                      |
| 5. ner_oncology_response_to_treatment       | 6. ner_oncology_posology                  | 7. ner_oncology                           | 8. ner_oncology_limited_80p_for_benchmarks |
| 9. ner_oncology_diagnosis                   | 10. ner_oncology_demographics             | 11. ner_oncology_biomarker                | 12. ner_oncology_anatomy_granular          |
| 13. ner_oncology_anatomy_general            |                                           |                                            |                                           |




- `ner_profiling_sdoh` :This pipeline can be used to explore all the available pretrained NER models at once for Social Determinants of Health. When you run this pipeline over your text, you will end up with the predictions coming out of each pretrained clinical NER model trained with embeddings_clinical.

|                 |                 |                 |                 |
| --------------- | --------------- | --------------- | --------------- |
| 1. ner_sdoh                                   | 2. ner_sdoh_social_environment_wip             | 3. ner_sdoh_mentions                             | 4. ner_sdoh_demographics_wip                    |
| 5. ner_sdoh_community_condition_wip           | 6. ner_sdoh_substance_usage_wip                | 7. ner_sdoh_access_to_healthcare_wip            | 8. ner_sdoh_health_behaviours_problems_wip     |
| 9. ner_sdoh_income_social_status_wip         |                                                 |                                                  |                                                  |


For more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb).









You can check [Models Hub](https://nlp.johnsnowlabs.com/models) page for more information about all these models and more.

In [ ]:
clinical_profiling_pipeline = PretrainedPipeline("ner_profiling_clinical", "en", "clinical/models")

ner_profiling_clinical download started this may take some time.
Approx size to download 2.9 GB
[OK!]


In [ ]:
text = '''A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .'''

In [ ]:
clinical_result = clinical_profiling_pipeline.fullAnnotate(text)[0]
clinical_result.keys()

dict_keys(['ner_vop_problem_wip_chunks', 'ner_sdoh_demographics_wip_chunks', 'ner_ade_clinical_chunks', 'ner_deid_augmented', 'ner_deid_subentity_augmented_i2b2', 'ner_posology_greedy_chunks', 'ner_vop_demographic_wip_chunks', 'ner_sdoh_community_condition_wip_chunks', 'ner_radiology_wip_clinical', 'ner_vop_slim_wip', 'ner_oncology_diagnosis_chunks', 'ner_vop_test_wip_chunks', 'ner_jsl_slim', 'ner_vop_wip', 'ner_risk_factors_chunks', 'jsl_ner_wip_clinical_chunks', 'ner_oncology_unspecific_posology_chunks', 'ner_vop_problem_reduced_wip', 'ner_vop_anatomy_wip', 'ner_vop_problem_reduced_wip_chunks', 'ner_eu_clinical_case_chunks', 'ner_oncology_test_chunks', 'ner_deid_synthetic', 'ner_oncology_posology_chunks', 'ner_oncology_tnm_chunks', 'ner_oncology_anatomy_general_chunks', 'ner_drugs_greedy', 'ner_abbreviation_clinical_chunks', 'ner_covid_trials_chunks', 'ner_human_phenotype_gene_clinical_chunks', 'ner_events_admission_clinical', 'jsl_ner_wip_greedy_clinical_chunks', 'ner_posology_greed

In [ ]:
import pandas as pd

def get_token_results(light_result):

  tokens = [j.result for j in light_result["token"]]
  sentences = [j.metadata["sentence"] for j in light_result["token"]]
  begins = [j.begin for j in light_result["token"]]
  ends = [j.end for j in light_result["token"]]
  model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

  df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

  for model_name in model_list:

    temp_df = pd.DataFrame(light_result[model_name])
    temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
    temp_df = temp_df[["jsl_label"]]

    # temp_df = get_ner_result(model_name)
    temp_df.columns = [model_name]
    df = pd.concat([df, temp_df], axis=1)

  return df

In [ ]:
get_token_results(clinical_result)

,sentence,begin,end,token,ner_deid_augmented,ner_deid_subentity_augmented_i2b2,ner_radiology_wip_clinical,ner_vop_slim_wip,ner_jsl_slim,ner_vop_wip,...,ner_events_clinical,ner_supplement_clinical,ner_oncology_unspecific_posology,ner_genetic_variants,ner_sdoh_wip,ner_radiology,ner_eu_clinical_case,ner_posology,ner_oncology_posology,ner_covid_trials
0,0,0,0,A,O,O,O,O,O,O,...,O,O,O,O,O,O,B-patient,O,O,O
1,0,2,12,28-year-old,O,O,O,O,B-Age,B-Age,...,O,O,O,O,B-Gender,O,I-patient,O,O,B-Gender
2,0,14,19,female,O,O,O,B-Gender,B-Demographics,B-Gender,...,O,O,O,O,B-Gender,O,I-patient,O,O,B-Gender
3,0,21,24,with,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
4,0,26,26,a,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0,409,416,appetite,O,O,I-Symptom,I-Symptom,I-Symptom,I-Symptom,...,I-PROBLEM,O,O,O,O,I-Symptom,I-clinical_condition,O,O,O
69,0,418,418,",",O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
70,0,420,422,and,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O
71,0,424,431,vomiting,O,O,B-Symptom,B-Symptom,B-Symptom,B-Symptom,...,B-PROBLEM,B-CONDITION,O,O,O,B-Symptom,B-clinical_condition,O,O,O


## NER Model Finder Pretrained Pipeline
`ner_model_finder`  pretrained pipeline trained with bert embeddings that can be used to find the most appropriate NER model given the entity name.

In [ ]:
from sparknlp.pretrained import PretrainedPipeline
finder_pipeline = PretrainedPipeline("ner_model_finder", "en", "clinical/models")

ner_model_finder download started this may take some time.
Approx size to download 148.7 MB
[OK!]


In [ ]:
result = finder_pipeline.fullAnnotate("oncology")[0]
result.keys()

dict_keys(['model_names'])

From the metadata in the 'model_names' column, we'll get to the top models to the given 'oncology' entity and oncology related categories.

In [ ]:
df= pd.DataFrame(zip(result["model_names"][0].metadata["all_k_resolutions"].split(":::"),
                     result["model_names"][0].metadata["all_k_results"].split(":::")),
                 columns=["category", "top_models"])

In [ ]:
df.head()

,category,top_models
0,oncology therapy,"['ner_jsl', 'jsl_rd_ner_wip_greedy_clinical', 'jsl_ner_wip_modifier_clinical', 'ner_jsl_enriched..."
1,clinical department,"['ner_jsl', 'jsl_rd_ner_wip_greedy_clinical', 'jsl_ner_wip_modifier_clinical', 'ner_events_clini..."
2,biomedical unit,['ner_clinical_trials_abstracts']
3,cancer genetics,['ner_cancer_genetics']
4,anatomy,"['ner_bionlp', 'ner_medmentions_coarse', 'ner_chexpert', 'ner_anatomy_coarse', 'ner_anatomy', 'n..."


## Resolver Pipelines

We have **Resolver pipelines** for converting clinical entities to their UMLS CUI codes. You will just feed your text and it will return the corresponding UMLS codes.



**Resolver Pipelines:**

|index|pipeline|Entity|Target
|-|-|-|:-:|
| 1|[umls_disease_syndrome_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/26/umls_disease_syndrome_resolver_pipeline_en_3_0.html)|Disease and Syndromes|UMLS CUI|
| 2|[umls_drug_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/26/umls_drug_resolver_pipeline_en_3_0.html)|Drug|UMLS CUI|
| 3|[umls_drug_substance_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/25/umls_drug_substance_resolver_pipeline_en_3_0.html)|Drug Substance|UMLS CUI|
| 4|[medication_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/09/01/medication_resolver_pipeline_en.html)|Drug|Adverse Reaction, RxNorm, UMLS, NDC, SNOMED CT|
| 5|[medication_resolver_transform_pipeline](https://nlp.johnsnowlabs.com/2022/09/01/medication_resolver_transform_pipeline_en.html)|Drug|Adverse Reaction, RxNorm, UMLS, NDC, SNOMED CT|
| 6|[umls_major_concepts_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/25/umls_major_concepts_resolver_pipeline_en_3_0.html)|Clinical Major Concepts| UMLS CUI|
| 7|[umls_clinical_findings_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/26/umls_clinical_findings_resolver_pipeline_en_3_0.html)|Clinical Findings|	UMLS CUI|
| 8|[icd9_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/09/30/icd9_resolver_pipeline_en.html)|Clinical Findings|ICD-9|
| 9|[atc_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/17/atc_resolver_pipeline_en.html)|Drug|ATC|
| 10|[cpt_procedures_measurements_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/17/cpt_procedures_measurements_resolver_pipeline_en.html)|Procedure, Measurement|CPT|
| 11|[hcc_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/17/hcc_resolver_pipeline_en.html)|Clinical Findings|HCC|
| 12|[hpo_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/17/hpo_resolver_pipeline_en.html)|Human Phenotype|HPO|
| 13|[cvx_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/10/12/cvx_resolver_pipeline_en.html)|Vaccine|CVX|
| 14|[icd10cm_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/11/02/icd10cm_resolver_pipeline_en.html)|Problem|ICD-10|
| 15|[icd10pcs_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/02/02/icd10pcs_resolver_pipeline_en.html)|Procedure|ICD-10-PCS|
| 16|[icdo_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/02/02/icdo_resolver_pipeline_en.html)|Problem|ICD-O|
| 17|[loinc_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/02/02/loinc_resolver_pipeline_en.html)|Test|LOINC|
| 17|[loinc_numeric_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/30/loinc_numeric_resolver_pipeline_en.html)|Test|LOINC|
| 18|[snomed_body_structure_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/17/snomed_body_structure_resolver_pipeline_en.html)|Clinical Findings|SNOMED|
| 19|[snomed_findings_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/17/snomed_findings_resolver_pipeline_en.html)|Clinical Findings|SNOMED CT|
| 20|[snomed_procedures_measurements_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/31/snomed_findings_resolver_pipelin_en.html)|Procedure, Test|SNOMED|
| 21|[mesh_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/25/mesh_resolver_pipeline_en.html)|Clinical Findings|MeSH|
| 22|[ndc_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/25/ndc_resolver_pipeline_en.html)|Drug|NDC|
| 23|[ncit_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/25/ncit_resolver_pipeline_en.html)|Clinical Findings|NCIt|
| 24|[rxcui_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/02/01/rxcui_resolver_pipeline_en.html)|Clinical Findings|RxCUI|
| 25|[hcpcs_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/30/hcpcs_resolver_pipeline_en.html)|Procedure|HCPCS|
| 26|[hgnc_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/30/hgnc_resolver_pipeline_en.html)|Gene|HGNC|
| 27|[icd10cm_generalised_resolver_pipeline](https://nlp.johnsnowlabs.com/2024/01/30/icd10cm_generalised_resolver_pipeline_en.html)|Clinical Findings|ICD-10-CM|
| 28|[abbreviation_pipeline](https://nlp.johnsnowlabs.com/2023/08/16/abbreviation_pipeline_en.html)|ABBR|Definitions and Categories|
| 29|[icd10cm_multi_mapper_pipeline](https://nlp.johnsnowlabs.com/2023/08/16/icd10cm_multi_mapper_pipeline_en.html)|ICD-10-CM|Billable Mappings, HCC Codes, Cause Mappings, Claim Mappings, SNOMED Codes, UMLS Codes and ICD-9 Codes|
| 30|[rxnorm_multi_mapper_pipeline](https://nlp.johnsnowlabs.com/2023/08/16/rxnorm_multi_mapper_pipeline_en.html)|RxNorm|Drug Brand Names, RxNorm Extension Brand Names, Action Mappings, Treatment Mappings, UMLS Codes, NDC Product Codes and NDC Package Codes|
| 31|[rxnorm_resolver_pipeline](https://nlp.johnsnowlabs.com/2023/08/16/rxnorm_resolver_pipeline_en.html)|Drug|RxNorm |
| 32|[snomed_multi_mapper_pipeline](https://nlp.johnsnowlabs.com/2023/08/16/snomed_multi_mapper_pipeline_en.html)|SNOMED Codes|ICD-10, ICD-O,UMLS |


**icd10cm_resolver_pipeline**

This pretrained pipeline extracts clinical conditions and map them to their corresponding ICD-10-CM codes. You’ll just feed your text and it will detect the related entities and return their corresponding ICD-10-CM codes.

In [ ]:
resolver_pipeline = PretrainedPipeline("icd10cm_resolver_pipeline", "en", "clinical/models")

text = """A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years and anisakiasis. Also, it was reported that fetal and neonatal hemorrhage"""

result = resolver_pipeline.fullAnnotate(text)[0]

icd10cm_resolver_pipeline download started this may take some time.
Approx size to download 3.3 GB
[OK!]


In [ ]:
import pandas as pd
chunks=[]
entities=[]
icd10cm_code=[]


for n,m in zip(result['chunk'], result['icd10cm']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    icd10cm_code.append(m.result)


df = pd.DataFrame({'chunks':chunks,
                   'entities':entities,
                   'icd10cm_code':icd10cm_code})

df

,chunks,entities,icd10cm_code
0,gestational diabetes mellitus,PROBLEM,O24.919
1,anisakiasis,PROBLEM,B81.0
2,fetal and neonatal hemorrhage,PROBLEM,P549


## Oncology Pipelines

**Oncology Pretrained Pipeline List:**


|index|model|index|model|
|-----:|:-----|-----:|:-----|
| 1| [oncology_biomarker_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_biomarker_pipeline_en.html)  | 2| [oncology_general_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_general_pipeline_en.html)  |
| 3| [oncology_therapy_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_therapy_pipeline_en.html)  | 4| [oncology_diagnosis_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_diagnosis_pipeline_en.html)  
| 5| [explain_clinical_doc_oncology](https://nlp.johnsnowlabs.com/2024/01/18/explain_clinical_doc_oncology_en.html)  |

In [ ]:
oncology_pipeline = PretrainedPipeline("oncology_biomarker_pipeline", "en", "clinical/models")

oncology_biomarker_pipeline download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [ ]:
oncology_pipeline.model.stages

[DocumentAssembler_dab6eeac879e,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_9d695d8e04cc,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_ecf280ca65e5,
 NerConverter_929f666beebc,
 MedicalNerModel_aeadb24f76a3,
 NerConverter_4f9b8da8c4c3,
 MedicalNerModel_eb9da4b9039b,
 NerConverter_40028785be7b,
 MedicalNerModel_299a97740594,
 NerConverter_a8bf552d0249,
 MERGE_acda18b976a6,
 MERGE_12af4df6fa27,
 ASSERTION_DL_8d77f383c928,
 ASSERTION_DL_163867728788,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 RelationExtractionModel_d0af74510daa,
 RelationExtractionModel_68ebe11369b6,
 RelationExtractionModel_513eb6317779]

In [ ]:
text = """Immunohistochemistry was negative for thyroid transcription factor-1 and napsin A. The test was positive for ER and PR, and negative for HER2."""

result = oncology_pipeline.fullAnnotate(text)[0]

result.keys()

dict_keys(['re_oncology_granular_wip', 'assertion_oncology_test_binary_wip', 're_oncology_wip', 'ner_oncology_biomarker_wip_chunk', 'ner_biomarker_chunk', 'ner_oncology_test_wip', 'document', 're_oncology_biomarker_result_wip', 'merged_chunk', 'ner_oncology_biomarker_wip', 'ner_biomarker', 'ner_oncology_test_wip_chunk', 'token', 'embeddings', 'pos_tags', 'assertion_oncology_wip', 'assertion_chunk', 'ner_oncology_wip', 'dependencies', 'ner_oncology_wip_chunk', 'sentence'])

**NER Results**

In [ ]:
chunks=[]
entities=[]
begins=[]
ends=[]
confidence=[]
for n in result['merged_chunk']:

    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity'])
    confidence.append(n.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities, 'confidence':confidence})

df

,chunks,begin,end,entities,confidence
0,Immunohistochemistry,0,19,Pathology_Test,0.9967
1,negative,25,32,Biomarker_Result,0.8323
2,thyroid transcription factor-1,38,67,Biomarker,0.296675
3,napsin A,73,80,Biomarker,0.64309996
4,positive,96,103,Biomarker_Result,0.8017
5,ER,109,110,Biomarker,0.948
6,PR,116,117,Biomarker,0.8711
7,negative,124,131,Biomarker_Result,0.8385
8,HER2,137,140,Oncogene,0.9359


**Assertion Status Results**

In [ ]:
chunks=[]
entities=[]
status=[]
confidence=[]

for n,m in zip(result['merged_chunk'],result['assertion_oncology_test_binary_wip']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)
    confidence.append(m.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

,chunks,entities,assertion,confidence
0,Immunohistochemistry,Pathology_Test,Medical_History,0.9926
1,negative,Biomarker_Result,Medical_History,0.9951
2,thyroid transcription factor-1,Biomarker,Medical_History,0.9951
3,napsin A,Biomarker,Medical_History,0.9926
4,positive,Biomarker_Result,Medical_History,0.9931
5,ER,Biomarker,Medical_History,0.9938


**Relation Extraction Results**

In [ ]:
result = oncology_pipeline.fullAnnotate(text)

rel_df = get_relations_df(result, 're_oncology_wip')

rel_df[rel_df.relation!= "O"]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
1,is_related_to,Biomarker_Result,25,32,negative,Biomarker,38,67,thyroid transcription factor-1,0.997901
2,is_related_to,Biomarker_Result,25,32,negative,Biomarker,73,80,napsin A,0.999566
3,is_related_to,Biomarker_Result,96,103,positive,Biomarker,109,110,ER,0.987820
4,is_related_to,Biomarker_Result,96,103,positive,Biomarker,116,117,PR,0.897783
8,is_related_to,Biomarker_Result,124,131,negative,Oncogene,137,140,HER2,0.986855


In [ ]:
result[0]['re_oncology_wip']

[Annotation(category, 0, 32, O, {'chunk2': 'negative', 'confidence': '0.97084755', 'entity2_end': '32', 'chunk1': 'Immunohistochemistry', 'entity1': 'Pathology_Test', 'entity2_begin': '25', 'chunk2_confidence': '0.8323', 'entity1_begin': '0', 'sentence': '0', 'direction': 'both', 'entity1_end': '19', 'entity2': 'Biomarker_Result', 'chunk1_confidence': '0.9967'}, []),
 Annotation(category, 25, 67, is_related_to, {'chunk2': 'thyroid transcription factor-1', 'confidence': '0.99790084', 'entity2_end': '67', 'chunk1': 'negative', 'entity1': 'Biomarker_Result', 'entity2_begin': '38', 'chunk2_confidence': '0.296675', 'entity1_begin': '25', 'sentence': '0', 'direction': 'both', 'entity1_end': '32', 'entity2': 'Biomarker', 'chunk1_confidence': '0.8323'}, []),
 Annotation(category, 25, 80, is_related_to, {'chunk2': 'napsin A', 'confidence': '0.9995658', 'entity2_end': '80', 'chunk1': 'negative', 'entity1': 'Biomarker_Result', 'entity2_begin': '73', 'chunk2_confidence': '0.64309996', 'entity1_beg